In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import copy as _copy
import pickle
import gzip
import os.path
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import scipy.optimize
import functools

<h1> Developing fitting the model for the beam based alignement measurement </h1>

<h2> Warning </h2>

All text below referes to the response matrix development. These 

<h2> Original procedure </h2>
The data have been treated and are available. Fits have been made to the different measuremnts. These revealed that the measurements can be explained with sufficient accuracy using second order polynoms. These fits are, however, model agnositic. 

The second step now aims to reach a model that fits the model to the measurement data

In this file different fit procedures are studied:
* Estimating the beta function from the model
* Fitting a second order scaling polynomial for the model
* Calculating reference data for each measurement assuming that the magnet transfer functions is not totally linear but contains a second order polynomial


In [ ]:
datetime.datetime.now().strftime('%Y %m %d')

<h2> Used modules </h2>

The different modules are collected in :mod:`bact2`.

* The response matrix directory collects the different module
* :mod:`reference_orbit` try to provide an *side effect free* interface to 
  ocelot next to orbit difference processing
* :mod:`commons` provides access to the preprocessed data. The current solution is a hack and will have to be adapted to the available data bases

* the preprocessed data are currently stored in a pickle file. The pickle file is created using 
`from_json_to_pickle`

In [ ]:
import bact2.applib.transverse_lib.bpm_data as bd
importlib.reload(bd)


In [ ]:
from bact2.applib.bba import model_fits, process_model_fits
from bact2.applib.bba.model_fit_funcs import min_func_adjust_2D
from bact2.applib.transverse_lib import reference_orbit
from bact2.applib.transverse_lib.process_model_fits import prepare_bpm_data

from bact2.applib.transverse_lib.distorted_orbit import closed_orbit_distortion
from bact2.applib.transverse_lib.distorted_orbit_process import orbit_distortion_for_kicker
importlib.reload(process_model_fits)

In [ ]:
import bact2.applib.bba.commons as commons
pickle_file_name = commons.pickle_file_name()

In [ ]:
with gzip.open(pickle_file_name) as fp:
    obj = pickle.load(fp)

In [ ]:
obj.original_dataframe.head()

In [ ]:
importlib.reload(reference_orbit)

<h2> The model </h2>
The model is currently wrapped in :mod:`reference_orbit`. It allows
* calculating the reference orbit
* creating a new model with a changed element
* calculating offset from a changed orbit to the reference orbit without messing with the original model...

The following lines are used to set up the orbit and to store the reference data

In [ ]:
default_orbit = reference_orbit.OrbitCalculator()

In [ ]:
moved_quadrupole_name = 'Q4M2D1R'

In [ ]:
moved_quadrupole = default_orbit.getElementbyName(moved_quadrupole_name)
moved_quadrupole, moved_quadrupole.dx, moved_quadrupole.dy

In [ ]:
new_quad, orbit = default_orbit.orbitCalculatorWithNewElement(name=moved_quadrupole_name, init_lattice=False)

In [ ]:
new_quad, new_quad.dx, new_quad.dy

In [ ]:
new_quad.dx = 0
new_quad.dy = 0


In [ ]:
orbit.initLattice()

In [ ]:
orbit_data_ref = orbit.orbitData()

In [ ]:
orbit_offset_filter = reference_orbit.OrbitOffset()
orbit_offset_filter.reference_data = orbit_data_ref

<h3> Setting up a model with a steerer at quadrupole position <h3>

In [ ]:
magnet_name = 'Q3M1T5R'
magnet_name = 'Q3M1T1R'
# magnet_name = 'Q3M1T6R'
#magnet_name = 'Q3M2D3R'
# magnet_name = moved_quadrupole_name

<h3> Selected quadrupole magnet  </h3>

Cuurrently only working with a single magnet. As soon it has been tested here a script will be developed for batch proessing all magnets

<h2> Measurement data </h2>

In [ ]:
def compute_polarity(value):
    if value == 0:
        return 0
    elif value > 0:
        return 1
    else:
        return -1

In [ ]:
line_colours = {0: 'k', 2: 'b'}
model_line_colours = {0: 'g', 2: 'c'}

def plot_style_for_value(value, colours=line_colours):
    aval = np.absolute(value)
    
    t_colour = colours[aval]
    
    polarity = compute_polarity(value)
    
    if polarity == 0:
        return t_colour + '-.'
    elif polarity == 1:
        return t_colour + '-'
    elif polarity -1:
        return t_colour + '--'
    
    raise AssertionError('Should not end up here')

<h2> Setting up the model </h2>

Fit offset together with kick

<h2> Approximation function using equivalent kicker and  beta function </h2>

In [ ]:
from bact2.applib.transverse_lib import  utils 
from bact2.applib.bba import distorted_orbit_process

In [ ]:
twiss = orbit.twissParameters()
twiss_df = utils.twiss_to_df(twiss)
twiss_df.head()

In [ ]:
twiss_bpm = orbit.twissParametersBpms()
twiss_bpm_df = utils.twiss_to_df(twiss_bpm)
twiss_bpm_df.head()

Currently starting with micro radians steering effects. Here the known magnet transfer functions should
be added. 

In [ ]:
machine_x, machine_y = distorted_orbit_process.machine_info_xy(twiss_df)
machine_bpm_x, machine_bpm_y = distorted_orbit_process.machine_info_xy(twiss_bpm_df)
# Bpm data do not contain the whole machine .... 
machine_bpm_x.Q = machine_x.Q
machine_bpm_y.Q = machine_y.Q

In [ ]:
models = distorted_orbit_process.MachineModelXY(
    x=distorted_orbit_process.MachineModel(orbit=machine_x, bpm=machine_bpm_x, kick=None),
    y=distorted_orbit_process.MachineModel(orbit=machine_y, bpm=machine_bpm_y, kick=None)
)

In [ ]:
guessed_angle=1e-5

In [ ]:
magnet_name

In [ ]:
kx, ky = distorted_orbit_process.model_kick(twiss_df, quadrupole_name=magnet_name, guessed_angle=guessed_angle)
kx

In [ ]:
co_x = orbit_distortion_for_kicker(models.x.bpm, kx)

In [ ]:
co_y = orbit_distortion_for_kicker(models.y.bpm, ky)

<h2> Trying to calculate kick closer to original data </h2>

In [ ]:
import bact2.pandas.dataframe.df_aggregate as dfg
from bact2.applib.transverse_lib.model_fits import select_bpm_data_in_model

In [ ]:
df = obj.original_dataframe[obj.original_dataframe.count_bpm_reads > 0]

In [ ]:
df_sel = df.loc[
    (df.mux_selector_selected == magnet_name) 
    #& (df.ramp_index.isin([0, 4, 12, 16]))
    ,
    :]
df_sel.shape

In [ ]:
bpm_data, bpm_data_m, bpm_data_rms, bpm_data_m_rms = prepare_bpm_data(df_sel, ref_row=0)

In [ ]:
# df.bpm_waveform_x_pos
dI = df_sel.mux_power_converter_setpoint

In [ ]:
def prepare_empty_fit_matrix(n_bpms, n_currents):
    A_prepared = np.zeros([n_bpms + 1, n_currents, n_bpms])
    for i in range(n_bpms):
        A_prepared[i+1, :, i] = 1
    return A_prepared

def prepare_fit_matrix(model_data, currents):
    n_bpms = len(model_data)
    n_currents = len(currents)

    A = prepare_empty_fit_matrix(n_bpms, n_currents)
    Ax = model_data[np.newaxis, :] * currents[:, np.newaxis]
    A[0, :, :] = Ax[:, :]
    return A

def fit_kicks(bpm_data, model_data, currents, bpm_scale_factor=1.0):
    A = prepare_fit_matrix(model_data, currents)
    
    bpm_data = bpm_data * bpm_scale_factor
    n_bpms = len(model_data)
    n_currents = len(currents)

    A = np.reshape(A, [n_bpms + 1, n_bpms * n_currents])
    b = np.reshape(bpm_data, [n_bpms * n_currents])
    r = scipy.optimize.lsq_linear(A.T, b)
    return r

In [ ]:
rx = fit_kicks(bpm_data_m.x, co_x, dI, bpm_scale_factor=1/1000.)
rx.x[0]

In [ ]:
ry = fit_kicks(bpm_data_m.y, co_y, dI, bpm_scale_factor=1/1000.)
ry.x[0]

In [ ]:
ref_mx =  ref_my = None
f = plt.figure(figsize=[20, 24])
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
dz_all = []
dz_model_all = []
for i in range(len(bpm_data_m.x)):
    row = df_sel.iloc[i, :]
    dI = df_sel.mux_power_converter_setpoint.iat[i]

    model_to_bpm = 1000

    if dI >= 0: 
        polarity = -1
    else:
        polarity =  1
    t_kick_scale_x = rx.x[0] * dI * polarity * model_to_bpm
    t_kick_scale_y = ry.x[0] * dI * polarity * model_to_bpm

    
    setpoint_round = int(dI)
           

    dx = bpm_data_m.x[i] - rx.x[1:] * model_to_bpm
    dy = bpm_data_m.y[i] - ry.x[1:] * model_to_bpm
    
    dx = dx * polarity
    dy = dy * polarity
    ds = bpm_data_m.s[i]
    ax1.plot(ds, dx, 'b', linewidth=.25, marker='x', label='bpm x')
    ax2.plot(ds, dy, 'r', linewidth=.25, marker='x', label='bpm y')
        
    ax1.plot(ds, co_x * t_kick_scale_x, 'c--', linewidth=.5, marker='+', label='kick x')
    ax2.plot(ds, co_y * t_kick_scale_y, 'm--', linewidth=.5, marker='+', label='kick y')

        
ax1.plot(ds, rx.x[1:] * model_to_bpm , 'k:', linewidth=.5, marker='x', label='offset x')
ax2.plot(ds, ry.x[1:] * model_to_bpm , 'k:', linewidth=.5, marker='x', label='offset y')

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('dx [mm]')
ax2.set_ylabel('dy [mm]')
ax1.legend(loc=0).set_draggable(True)
ax2.legend(loc=0).set_draggable(True)


In [ ]:
prepare_bpm_data?